<a href="https://colab.research.google.com/github/HSS107048212/hoop4s/blob/main/%E6%BC%82%E6%9B%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gspread-dataframe

In [2]:
pip install pygsheets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 1.6 MB/s eta 0:00:00


In [3]:
import os
import re
import smtplib
import pandas as pd
import gspread

from datetime import datetime
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
#from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Remaining code has been shortened for brevity...

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import numpy as np
import pygsheets

In [14]:
HOME_PATH = '/content/' # path of Colab for testing

# Authorization
gc = pygsheets.authorize(service_file=HOME_PATH+'orderdata-copyto-parttime-5c19ba8fcba4.json')

# Google sheet URL and authentication
Order_SPREADSHEET_rec = 'https://docs.google.com/spreadsheets/d/14G7aVadVpLyJPIcTVps51Uwb5EAxc1uErTBfGoCA0hw/edit?usp=sharing' # 需要儲存為Google試算表的格式，而不能是直接上傳的excel。
sht_rec = gc.open_by_url(Order_SPREADSHEET_rec)
wks_rec = sht_rec.worksheet_by_title("第 1 张表单回复")

In [15]:
df_rec = wks_rec.get_as_df()
df_rec

,时间戳记,电子邮件地址,姓名或暱稱,書名,作者\n若有多位作者，則填寫第一位，例如：余華等人,漂書序號,「取書」日期,請用一句話表達您的感謝之情吧，我們會幫您轉達給贈書之人。（非必填）,對於本活動您有什麼建議嗎？我們期待您的回饋並加以改進服務。（非必填）
0,11/27/2023 16:32:39,wufengcn001@gmail.com,吳鋒,小國大想象,鄭志凱,O01230003,2023/11/27,非常感謝你的書！祝你生活順利～,
1,11/27/2023 16:32:39,wufengcn001@gmail.com,吳鋒,小國大想象(測試),鄭志凱,O01230001,2023/11/27,非常感謝你的書！祝你生活順利～,


In [16]:
# Google sheet URL and authentication
Order_SPREADSHEET_pro = 'https://docs.google.com/spreadsheets/d/1Z4QcCfefF_7_NrWySYSEUvaoPPcFRGsS4hbYEteUaeM/edit?usp=sharing' # 需要儲存為Google試算表的格式，而不能是直接上傳的excel。
sht_pro = gc.open_by_url(Order_SPREADSHEET_pro)
wks_pro = sht_pro.worksheet_by_title("第 1 张表单回复")

In [17]:
df_pro = wks_pro.get_as_df()
df_pro

,时间戳记,电子邮件地址,姓名或暱稱,「漂書」日期\n請填入您將書籍放到取物站的日期,書名,作者\n若有多位作者，則填寫第一位，例如：余華等人,漂書序號,請問您是否願意提供您的個人資訊，讓取書者聯繫您表達感謝？ [我的姓名或暱稱],請問您是否願意提供您的個人資訊，讓取書者聯繫您表達感謝？ [我的 email 或其他聯繫資訊],每本書都將在合作社展示 30 天，若期限過後，我想要,請用一句話介紹這本書籍吧~ （非必填）\n不論是您的心得、好句摘錄，都可以讓新主人有機會認這本好書。,如果可以的話，可以提供您的書籍照片嗎？（非必填）,對於本活動您有什麼建議嗎？我們期待您的回饋並加以改進服務。（非必填）
0,11/27/2023 16:17:21,ubo11185@gapp.nthu.edu.tw,陳皆錫,2023/11/27,小國大想像,鄭志凱,O01230003,願意提供,願意提供,將書捐贈給合作社,「創新創業，以小博大的巧實力」,https://drive.google.com/open?id=1wD426Q-JI_CU...,


In [18]:
# Extract the '漂書序號' column from both dataframes
book_givers_serials = df_pro['漂書序號']
book_receivers_serials = df_rec['漂書序號']

# Compare the two lists to find common serial numbers
common_serials = set(book_givers_serials).intersection(set(book_receivers_serials))

# Convert the result to a list and sort it
common_serials_list = sorted(list(common_serials))
common_serials_list

['O01230003']

In [31]:
import smtplib
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

from datetime import datetime

current_month = datetime.now().month

# Email configuration
os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
os.environ["COOP_PASSWORD"] = "jwsscmoiffdyukmm"
# Get sender email and password from environment variables
sender_email = os.getenv("COOP_EMAIL")
sender_password = os.getenv("COOP_PASSWORD")


In [35]:
# Correcting the issue with the column name that contains newline character
# We need to use the exact column name without splitting it with a backslash

# Finding the correct column name for book introduction
book_intro_col = [col for col in df_pro.columns if '請用一句話介紹這本書籍吧' in col][0]
thank_intro_col = [col for col in df_rec.columns if '請用一句話表達您的感謝之情吧' in col][0]

# Iterate through each common serial number and print the corresponding book introduction
for serial in common_serials_list:
    # Filter rows where the serial number matches
    matching_rows_pro = df_pro[df_pro['漂書序號'] == serial]

    # Print the book introduction for each matching row
    for _, row in matching_rows_pro.iterrows():
        print(f"漂書序號: {serial}")
        print(f"贈書人介紹此書: {row[book_intro_col]}")
        print(f"贈書人暱稱: {row['姓名或暱稱']}")
        print(f"贈書人電子郵件: {row['电子邮件地址']}\n")
        book_name=row['書名']
        receiver_email_pro=row['电子邮件地址']
        receiver_name_pro=row['姓名或暱稱']
        receiver_intro_pro=row[book_intro_col]

    matching_rows_rec = df_rec[df_rec['漂書序號'] == serial]
    # Print the book introduction for each matching row
    for _, row in matching_rows_rec.iterrows():
        print(f"漂書序號: {serial}")
        print(f"受贈人回應: {row[thank_intro_col]}")
        print(f"受贈人暱稱: {row['姓名或暱稱']}")
        print(f"受贈人電子郵件: {row['电子邮件地址']}\n")
        receiver_email_rec=row['电子邮件地址']
        receiver_name_rec=row['姓名或暱稱']
        receiver_intro_rec=row[thank_intro_col]

    # Email content to pro
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email_pro
    message['Subject'] = f'十里方圓漂書站贈書:贈書《{book_name}》已領取'

    email_content = f"Dear {receiver_name_pro},\n"
    email_content += f"===================================\n"
    email_content += f"您捐贈的書籍: 《{book_name}》 已被領取\n"
    email_content += f"===================================\n"
    email_content += f"取書者回覆您: {receiver_intro_rec} \n"

    message.attach(MIMEText(email_content, "plain"))

    # Connect to the SMTP server and send the email
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(message)

    # Email content to rec
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email_rec
    message['Subject'] = f'十里方圓漂書站贈書:贈書者分享《{book_name}》'

    email_content = f"Dear {receiver_name_rec},\n"
    email_content += f"===================================\n"
    email_content += f"您領取的書籍:《{book_name}》附贈書者留言\n"
    email_content += f"===================================\n"
    email_content += f"贈書者: {receiver_intro_pro} \n"

    message.attach(MIMEText(email_content, "plain"))

    # Connect to the SMTP server and send the email
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(message)

漂書序號: O01230003
贈書人介紹此書: 「創新創業，以小博大的巧實力」
贈書人暱稱: 陳皆錫
贈書人電子郵件: ubo11185@gapp.nthu.edu.tw

漂書序號: O01230003
受贈人回應: 非常感謝你的書！祝你生活順利～
受贈人暱稱: 吳鋒
受贈人電子郵件: wufengcn001@gmail.com

